In [22]:
# Import the random forest package
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report
from sklearn import preprocessing
from time import time

# Create the random forest object which will include all the parameters
# for the fit
forest = RandomForestClassifier(n_estimators = 100)

data_import = pd.read_csv('/home/demos/hr-analytics/data/raw_data.csv')


       satisfaction_level  last_evaluation  number_projects  \
count        14999.000000     14999.000000     14999.000000   
mean             0.612834         0.716102         3.803054   
std              0.248631         0.171169         1.232592   
min              0.090000         0.360000         2.000000   
25%              0.440000         0.560000         3.000000   
50%              0.640000         0.720000         4.000000   
75%              0.820000         0.870000         5.000000   
max              1.000000         1.000000         7.000000   

       average_monthly_hours  time_spent_company  work_accident          left  \
count           14999.000000        14999.000000   14999.000000  14999.000000   
mean              201.050337            3.498233       0.144610      0.238083   
std                49.943099            1.460136       0.351719      0.425924   
min                96.000000            2.000000       0.000000      0.000000   
25%               156.00000

In [36]:

data = data_import.loc[data['last_evaluation'] >= 0.3]

data['left'] = data['left'].apply(lambda x : 'yes' if x == 1 else 'no')

le_sales = preprocessing.LabelEncoder()
le_sales.fit(data["department"])
data["department"] = le_sales.transform(data.loc[:,('department')])

le_salary_level = preprocessing.LabelEncoder()
le_salary_level.fit(data["salary_level"])
data["salary_level"] = le_salary_level.transform(data.loc[:,('salary_level')])

still_here = data.loc[data['left'] == 'no']
already_left = data.loc[data['left'] == 'yes']

X = data[["satisfaction_level", "last_evaluation", "number_projects", "average_monthly_hours", "time_spent_company", 
                 "work_accident", "promotion_last_5_years", "department", "salary_level"]]
still_here_X = still_here[["satisfaction_level", "last_evaluation", "number_projects", "average_monthly_hours", "time_spent_company", 
                 "work_accident", "promotion_last_5_years", "department", "salary_level"]]
already_left_X = already_left[["satisfaction_level", "last_evaluation", "number_projects", "average_monthly_hours", "time_spent_company", 
                 "work_accident", "promotion_last_5_years", "department", "salary_level"]]


y = data["left"]
still_here_y = still_here["left"]   
already_left_y = already_left["left"]   
                
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

print "len train set ",len(X_train)
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

# Fit the training data to the Survived labels and create the decision trees
clf = RandomForestClassifier(n_estimators=25, n_jobs=6)

# use a full grid over all parameters
param_grid = {"max_depth": [None],
              "max_features": [3],
              "min_samples_split": [3],
              "min_samples_leaf": [1],
              #"bootstrap": [True, False],
              "criterion": ["entropy"]
             }

#forest = clf.fit(X_train, y_train)
grid_search = GridSearchCV(clf, param_grid=param_grid)

start = time()
print start
forest = œœ_train, y_train)
print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))


report(grid_search.cv_results_)

y_true, y_pred = still_here_y, forest.predict(still_here_X)


df_compare = pd.DataFrame()
df_compare['y_true'] = y_true
df_compare = df_compare.reset_index()
df_compare['y_pred'] = y_pred
print df_compare

df_compare.to_csv("/home/demos/hr-analytics/output/compare_df.csv")
print(classification_report(y_true, y_pred))
    

output_proba = forest.predict_proba(still_here_X)

output_proba_df = pd.DataFrame.from_records(output_proba)
output_proba_df.sort_values(by=1, ascending=False, inplace=True)
output_proba_df.to_csv("/home/demos/hr-analytics/output/prediction_proba.csv")



output = forest.predict(X_test)
output_df = pd.Series(output)


#scores = cross_val_score(forest, still_here_X, still_here_y, cv = 10, scoring='precision')


print "finished"





len train set  8999
1480956279.38
GridSearchCV took 1.56 seconds for 1 candidate parameter settings.
Model with rank: 1
Mean validation score: 1.000 (std: 0.000)
Parameters: {'max_features': 3, 'min_samples_split': 3, 'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1}

       index y_true y_pred
0       2000     no    yes
1       2001     no    yes
2       2002     no    yes
3       2003     no    yes
4       2004     no    yes
5       2005     no    yes
6       2006     no    yes
7       2007     no    yes
8       2008     no    yes
9       2009     no    yes
10      2010     no    yes
11      2011     no    yes
12      2012     no    yes
13      2013     no    yes
14      2014     no    yes
15      2015     no    yes
16      2016     no    yes
17      2017     no    yes
18      2018     no    yes
19      2019     no    yes
20      2020     no    yes
21      2021     no    yes
22      2022     no    yes
23      2023     no    yes
24      2024     no    yes
25      2025 

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

         no       0.00      0.00      0.00     11428
        yes       0.00      0.00      0.00         0

avg / total       0.00      0.00      0.00     11428



KeyError: 1